# Clone Groups
*Jupyter Notebook to clone groups and their items. Configurable notebook over this awesome [`clone_items`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html#arcgis.gis.ContentManager.clone_items) function*

In [ ]:
# common imports
from arcgis.gis import GIS

***Note**, if you are unable to import local `partnerutils`, **copy the following functions** from [`clone_utils`](https://github.com/mpayson/esri-partner-tools/blob/master/partnerutils/clone_utils.py)

In [ ]:
from partnerutils.clone_utils import search_group_title, search_item_title, clone_items_modify

## User Input

### GIS Configuration
Parameter information [here](https://developers.arcgis.com/python/guide/using-the-gis/).
* **source** - the ArcGIS Online organization that contains the default groups & items
* **target** - the new organization

In [ ]:
# log in to respective portals
source = GIS("<template GIS>", "<un>", "<pw>")
target = GIS("<project GIS>", "<un>", "<pw>")

### Clone Configuration
Defines the groups that will be cloned and the naming structures of cloned items & groups

In [ ]:
# groups to be copied from source to target organization
GROUPS = ["<GROUP1>", "<GROUP2>"]

# (optional) new item name structure, {} replaced by source item name
NAME_TEMPLATE = "<APP NAME> {}"
# (optional) user folder for new items
FOLDER = "<APP NAME> Content" 

## Execution

### Modify Functions
Sometimes you don't want a 1:1 copy, these functions will modify the cloned item properties. A few notes:
* `modify_item_callback` receives the cloned item and its gis. It should return a flattened dict of properties [here](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html?highlight=clone_items#arcgis.gis.Item.update)
* `modify_group_callback` receives the cloned group, its expected title, and its gis. It should return a dict of properties [here](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html?highlight=clone_items#arcgis.gis.Group.update)
* The default behavior is to update the item to match the NAME_TEMPLATE, if it's not defined these will not be called

In [ ]:
# update title functions
def modify_item_callback(item, target_gis):
    title = NAME_TEMPLATE.format(item.title)
    while search_item_title(target_gis, title):
        title = input("Title `{0}` for ITEM `{1}` already exists \n new title: "
                         .format(title, item.title))
    return {"title": title}
def modify_group_callback(group, expected_title, target_gis):
    title = NAME_TEMPLATE.format(expected_title)
    while search_group_title(target_gis, title):
        title = input("Title `{0}` for GROUP `{1}` already exists \n new title: "
                         .format(title, expected_title))
    return {"title": title}

# these functions provide little utility if NAME_TEMPLATE is None
# so don't use if this is the case
modify_item = modify_item_callback if NAME_TEMPLATE else None
modify_group = modify_group_callback if NAME_TEMPLATE else None

### Clone

In [ ]:
# get groups
source_groups = [search_group_title(source, title) for title in GROUPS]

# copy the groups
results = clone_items_modify(source_groups, target,
                                     modify_item_callback=modify_item,
                                     modify_group_callback=modify_group,
                                     copy_data=False, search_existing_items=False, folder=FOLDER)